In [192]:
import os
from absbox import mkDeal, API, EnginePath
from absbox.local.component import *
from absbox.local.base import *
import datetime
import pandas as pd 
localAPI = API(EnginePath.LOCAL, lang="english",check=False)

name = "Deal"
dates = {
    "collect": [
        "2024-08-15",
        "2024-08-31",
    ],  # collection_dates: next pool colle
    "pay": [
        "2024-08-15",  # payment_dates last distribution payment date, next
        "2024-08-31",
    ],
    "stated": "2028-06-21",
    "poolFreq": "MonthEnd",
    "payFreq": "MonthEnd",
}

pool = {
    "assets": [['Mortgage', {'originBalance': 4852735.68, 'originRate': ['fix', 0.25713118105449384], 'originTerm': 5, 'freq': 'Monthly', 'type': 'Level', 'originDate': '2024-06-28'}, 
               {'currentBalance': 4852735.68, 'currentRate': 0.25713118105449384, 'remainTerm': 3, 'status': 'Current'}]],
    "issuanceStat": {"IssuanceBalance": 5_068_878.19
                     },
}
accounts = {
            "collections_interest": {"balance": 0},
            "collections_principal": {"balance": 0},
            "collections_account": {"balance": 496702.51},
            "revolBuyAcc": {"balance": 0},
            "reserveAccount": {
                "balance": 21_440.31,
                "type": {
                    "targetReserve": [
                        (
                            "bondBalance",
                            "A_GBP",
                        ),
                        0.005,
                    ]
                },
            },
        }

                
bonds = {
            "A":{"A1": {
                "balance": 4_261_920.04,
                "rate": 0.12,
                "originBalance": 10_000_000,
                "originRate": 0.12,
                "startDate": "2023-02-21",
                "rateType": {
                    "floater": [0.0495, "SONIA", 0.12, "MonthEnd"],
                    "dayCount": "DC_ACT_360",
                },
                "maturityDate": "2025-06-21",
                "bondType": {"Lockout": "2024-02-21"},
            },},
            "EQ": {
                "balance": 473_546.67,
                "rate": 0.0,
                "originBalance": 1,
                "originRate": 0.0,
                "startDate": "2023-02-21",
                "maturityDate": "2025-06-21",
                "rateType": {"Fixed": 0.00},
                "bondType": {"Equity": None},
            },
        }

fees = {
    "technology_fee": {"type": {"annualPctFee": [("bondBalance","A"),0.0005]}},
    "structuring_fee": {
        "type": {"annualPctFee": [("originalBondBalance",), 0.0070], "feeDue": 70000},
    },
    "servicing_fee": {"type": {"annualPctFee": [("bondBalance", "A",), 0.01]}},
   "uncommited_fee": {"type": {"annualPctFee": [("excess",("originalBondBalance", "A"),("bondBalance", "A"),),0.01],
    "feeStart": "2024-02-21"}},
    "security_fee": {"type": {"byPeriod": 2142.86}},
}
# fees = create_spv_fees(self.static_data['spv_fees_static'])
waterfall = {
            "revolving":[
                                ["calcIntByGroup",["A"]],                               
                                ["transfer", "collections_interest", "collections_account"],
                                ["transfer", "collections_principal", "collections_account"],
                                ["calcAndPayFee","collections_account",["technology_fee","structuring_fee","security_fee","uncommited_fee",],],
                                ["accrueAndPayIntByGroup","collections_account","A","byStartDate"],
                                ["calcAndPayFee", "collections_account", ["servicing_fee"]],
                                ["transfer", "collections_account", "revolBuyAcc",
                                                                                {"formula": ("-", ("bondBalance", "A"),
                                                                                ("factor", ("sum", 
                                                                                            ("poolBalance",),
                                                                                            ("accountBalance","collections_interest","collections_principal","collections_account","revolBuyAcc","reserveAccount"), ), 0.9,))}],
                                ["transfer","collections_account","reserveAccount",{"reserve": "gap"},],
                                
                                ["transfer", "collections_account", "revolBuyAcc"],
                                ["buyAsset",["Current|Defaulted", 1.0, 0],"revolBuyAcc",None,],],
                    "amortizing":[
                                ["transfer", "collections_interest", "collections_account"],
                                ["transfer", "collections_principal", "collections_account"],
                                ["transfer", "reserveAccount", "collections_account"],
                                ["calcIntByGroup",["A"]],  # actions if deal is in Amortizing status
                                ["calcAndPayFee","collections_account",["technology_fee","structuring_fee","security_fee","uncommited_fee",],],
                                ["accrueAndPayIntByGroup","collections_account","A","byStartDate"],
                                ["payPrinByGroup","collections_account","A","byStartDate"],
                                ["calcAndPayFee", "collections_account", ["servicing_fee"]],
                                ["payIntResidual", "collections_account", "EQ", {"limit":{"balCapAmt": 20000}}],
                                ["payPrin", "collections_account", ["EQ"]],
                                ],

            "cleanUp": [
               ["sellAsset", ["Current|Defaulted", 1.0, 0], "collections_account"],
                ["transfer", "reserveAccount", "collections_account"],
                ["transfer", "revolBuyAcc", "collections_account"],
                ["accrueAndPayIntByGroup","collections_account","A","byStartDate"],
                ["payIntResidual", "collections_account", "EQ", {"limit":{"balCapAmt": 20000}}],
                ["transfer", "collections_interest", "collections_account"],
                ["payPrinByGroup","collections_account","A","byStartDate"],
                ["payPrin", "collections_account", ["EQ"]],  
            ],
            "Defaulted": [
               ["sellAsset", ["Current|Defaulted", 1.0, 0], "collections_account"],
                ["transfer", "reserveAccount", "collections_account"],
                ["transfer", "revolBuyAcc", "collections_account"],
                ["accrueAndPayIntByGroup","collections_account","A","byStartDate"],
                ["payIntResidual", "collections_account", "EQ", {"limit":{"balCapAmt": 20000}}],
                ["transfer", "collections_interest", "collections_account"],
                ["payPrinByGroup","collections_account","A","byStartDate"],
                ["payPrin", "collections_account", ["EQ"]],  
            ],
        }
collects = [
            ["CollectedPrincipal", "collections_principal"],
            ["CollectedRecoveries", "collections_principal"],
            ["CollectedPrepayment", "collections_principal"],
            ["CollectedInterest", "collections_interest"],
        ]

trigger = {
            "BeforeDistribution": {
                "amortization_trigger": {
                    "condition": ["any", [">=", "2025-05-16"]],
                    "effects": ("newStatus", "Amortizing"),
                    "status": False,
                    "curable": False,
                },
                "LTV_trigger": {
                    "condition": [
                        (
                            "/",
                            ("bondBalance","A"),
                            ("sum", ("poolBalance",),
                             ("accountBalance","collections_interest","collections_principal","collections_account","revolBuyAcc","reserveAccount"), ),
                        ),
                        ">",
                        0.9,
                    ],
                    "effects": ("newStatus", "Defaulted"),
                    "status": False,
                    "curable": True,
                },
                "DR_trigger": {
                    "condition": ["all", 
                          ["<=", "2025-06-30"],
                          [">=", "2024-12-31"],
                          [(("ratio",("cumPoolCollection", None, "Defaults"),("curPoolCollectionStats", None, -4, "BegBalance"))),
                        ">",
                        0.028,],
                        ],
                    "effects": ("newStatus", "Defaulted"),
                    "status": False,
                    "curable": False,
                },
            }
        }

deal_data = {
            "name": name,
            "dates": dates,
            "pool": pool,
            "accounts": accounts,
            "fees": fees,
            "bonds": bonds,
            "waterfall": waterfall,
            "collect": collects,
            "trigger": trigger,
            "status": "Revolving",
        }

deal = mkDeal(deal_data)

### For some reason the last Revol asset gives us a non negative pool cashflow
revol = [
     ['Mortgage', {'originBalance': 50107.574603659625, 'originRate': ['fix', 0.25707101434409524], 'originTerm': 1, 'freq': 'Monthly', 'type': 'Level', 'originDate': '2024-01-13'}, {'currentBalance': 20000.7559048708, 'currentRate': 0.25707101434409524, 'remainTerm': 1, 'status': 'current'}], 
     ['Mortgage', {'originBalance': 36967.51046002586, 'originRate': ['fix', 0.2557650305094591], 'originTerm': 5, 'freq': 'Monthly', 'type': 'Level', 'originDate': '2024-01-13'}, {'currentBalance': 10000.23953829672, 'currentRate': 0.2557650305094591, 'remainTerm': 3, 'status': 'current'}], 
     ['Mortgage', {'originBalance': 32715.948443960933, 'originRate': ['fix', 0.25526498158340144], 'originTerm': 6, 'freq': 'Monthly', 'type': 'Level', 'originDate': '2024-01-13'}, {'currentBalance': 32246.528814500976, 'currentRate': 0.25526498158340144, 'remainTerm': 3, 'status': 'current'}]
]

# revol = [['Mortgage', {'originBalance': 50107.574603659625, 'originRate': ['fix', 0.25707101434409524], 'originTerm': 1, 'freq': 'Monthly', 'type': 'Level', 'originDate': '2024-01-13'}, {'currentBalance': 49144.7559048708, 'currentRate': 0.25707101434409524, 'remainTerm': 1, 'status': 'current'}], 
# ['Mortgage', {'originBalance': 36967.51046002586, 'originRate': ['fix', 0.2557650305094591], 'originTerm': 5, 'freq': 'Monthly', 'type': 'Level', 'originDate': '2024-01-13'}, {'currentBalance': 35598.23953829672, 'currentRate': 0.2557650305094591, 'remainTerm': 3, 'status': 'current'}],]

#revol = [['Mortgage', {'originBalance': 50107.574603659625, 'originRate': ['fix', 0.25707101434409524], 'originTerm': 1, 'freq': 'Monthly', 'type': 'Level', 'originDate': '2024-01-13'}, {'currentBalance': 49144.7559048708, 'currentRate': 0.25707101434409524, 'remainTerm': 1, 'status': 'current'}],]



revolving = (
    "revolving",
    ["constant", *revol],
    ("Pool", ("Mortgage", {"CDR": 0}, {"CPR": 0}, None, None), None, None),
)

r = localAPI.run(
            deal,
            poolAssump=(
                "Pool",
                ("Mortgage", {"CDR": 0}, {"CPR": 0}, None, None),
                None,
                None,
            ),
            runAssump=[
                ("interest", ("SONIA", 0.0519), ("EURIBOR1M", 0.0858)),
                revolving,
                ("call", {"afterDate": "2025-06-01"}),
                #("stop","2024-10-20")
            ],
            read=True,
        )


Connecting engine server -> http://localhost:8081

✅Connected, local lib:0.28.7, server:0.29.6

Warning Message from server:
Bond A1 is not paid off
Bond EQ is not paid off
Account collections_principal has cash to be distributed

In [193]:
r['pool']['flow']

,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,BorrowerNum,PrepayPenalty,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,,,,
2024-08-31,4852735.68,0.00,0.00,0,0,0,0,0.257131,None,None,0.00,0,0,0,0,0
2024-09-30,146020.38,1729428.92,107107.60,0,0,0,0,0.257071,None,None,1729428.92,0,0,0,0,0
2024-10-31,565246.76,2283236.64,88724.55,0,0,0,0,0.256171,None,None,4012665.56,0,0,0,0,0
2024-11-30,909299.02,2144418.15,65228.31,0,0,0,0,0.255438,None,None,6157083.71,0,0,0,0,0
2024-12-31,406392.06,502906.96,19351.41,0,0,0,0,0.142297,None,None,6659990.67,0,0,0,0,0
2025-01-31,0.00,406392.06,8648.71,0,0,0,0,0.000000,None,None,7066382.73,0,0,0,0,0


In [197]:
r['pool']['flow']

,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,BorrowerNum,PrepayPenalty,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,,,,
2024-08-31,4852735.68,0.00,0.00,0,0,0,0,0.257131,None,None,0.00,0,0,0,0,0
2024-09-30,146020.38,1729428.92,107107.60,0,0,0,0,0.257071,None,None,1729428.92,0,0,0,0,0
2024-10-31,565246.76,2283236.64,88724.55,0,0,0,0,0.256171,None,None,4012665.56,0,0,0,0,0
2024-11-30,909299.02,2144418.15,65228.31,0,0,0,0,0.255438,None,None,6157083.71,0,0,0,0,0
2024-12-31,406392.06,502906.96,19351.41,0,0,0,0,0.142297,None,None,6659990.67,0,0,0,0,0
2025-01-31,0.00,406392.06,8648.71,0,0,0,0,0.000000,None,None,7066382.73,0,0,0,0,0


In [194]:
4852735.68

4852735.68

In [196]:
1729428.92+146020.38 - 4852735.68

-2977286.38

In [186]:
80132.02+24851.38

104983.40000000001

In [174]:
3269327.14+1729428.92 - 454453.13+3269327.14

7813630.07

In [161]:
r['triggers']['BeginDistributionWF']['amortization_trigger']

,status,memo
date,,
2024-08-31,False,<Tag:Any:2024-08-31 GE 2025-05-16>
2024-09-30,False,<Tag:Any:2024-09-30 GE 2025-05-16>


In [162]:
r['result']['status']

,Date,From,To
0,2024-10-20,DealEnd,Stop Run Flag


In [163]:
r['accounts']['revolBuyAcc']

,balance,change,memo
date,,,
2024-08-31,0.00,0.00,"<TransferBy:collections_account,revolBuyAcc,DS..."
2024-08-31,454453.14,454453.14,"<Transfer:collections_account,revolBuyAcc>"
2024-08-31,0.00,-454453.14,"<PurchaseAsset:Consol,454453.13>"
2024-09-30,0.00,0.00,"<TransferBy:collections_account,revolBuyAcc,DS..."
2024-09-30,1759193.93,1759193.93,"<Transfer:collections_account,revolBuyAcc>"
2024-09-30,0.00,-1759193.93,"<PurchaseAsset:Consol,1759193.92>"


In [164]:
r['accounts']['collections_account'].loc['2025-05-30':]

,balance,change,memo
date,,,
